<a href="https://colab.research.google.com/github/taskswithcode/probability_for_ml_notebooks/blob/main/ProbForML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook for Probability for Machine Learning - Part 2**

This is a notebook link for the video [Probability for Machine Learning - 2](https://youtu.be/LOh5-LTdosU)

### Example True and Predicted distribution from video


In [ ]:
y_true = [0.9, 0.1] #True distribution

In [ ]:
y_pred = [0.8, 0.2] #Predicted distribution

### 1.  Cross entropy function

In [ ]:
import numpy as np

def cross_entropy(y_true, y_pred):
    """
    Calculate the cross entropy between a true and a predicted distribution

    Args:
    y_true: A list representing the true probability distribution.
    y_pred: A list representing the predicted probability distribution.

    Returns:
    The cross entropy of the two distributions.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # The distributions should be valid probability distributions
    assert np.isclose(np.sum(y_true), 1), "True distribution should sum to 1."
    assert np.isclose(np.sum(y_pred), 1), "Predicted distribution should sum to 1."
    assert (y_true >= 0).all(), "All elements in the true distribution should be non-negative."
    assert (y_pred >= 0).all(), "All elements in the predicted distribution should be non-negative."

    # Exclude zero values to avoid log(0)
    mask = y_true > 0
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    return round(-np.sum(y_true * np.log2(y_pred)),3)

### 3. Compute Cross entropy

In [ ]:
print(y_true)
print(cross_entropy(y_true, y_true)) #Note this is the lowest value we can get for the entropy of the distribution [.9,.1]

[0.9, 0.1]
0.469


In [ ]:
for i in range(1,9):
    i = round(i*.1,1)
    y_pred = [round(y_true[0] - i,1), round(y_true[1] + i,1)]
    print(y_pred)
    print(cross_entropy(y_true, y_pred)) #All the cross entropy values will be more than the number computed above. So minimizing cross entropy loss enables a model to learn

[0.8, 0.2]
0.522
[0.7, 0.3]
0.637
[0.6, 0.4]
0.795
[0.5, 0.5]
1.0
[0.4, 0.6]
1.263
[0.3, 0.7]
1.615
[0.2, 0.8]
2.122
[0.1, 0.9]
3.005
